In [2]:
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
import threading
import cv2
import os
import time 
import numpy as np
from align_faces import warp_and_crop_face, get_reference_facial_points
from mtcnn.detector import MtcnnDetector
detector = MtcnnDetector()


In [125]:
import torch
print(torch.__version__)
torch.cuda.is_available ()

1.11.0+cpu


False

In [63]:
def mask_detect(image):
    (h,w) = image.shape[:2]
    boxes, facial5points = detector.detect_faces(image)
    face = np.zeros(shape =(256,256,3))
    if(len(boxes)!=0):
        for box in boxes:
            (startX,startY,endX,endY)=box[:4].astype('int')
            #ensure the bounding boxes fall within the dimensions of the frame
            (startX,startY)=(max(0,startX),max(0,startY))
            (endX,endY)=(min(w-1,endX), min(h-1,endY))
            #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
            c=cv2.resize(image[startY:endY, startX:endX],(256,256))
            face = c
            color = (255,0,0)
            cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
    return image, face

In [42]:
len_of_lst = 128 + 128
print(len_of_lst)

256


In [37]:
face = np.zeros(shape =(256,256,3))
print(face.shape)

(256, 256, 3)


In [ ]:
len_of_lst = 192
def PlayCamera(id):    
    num_of_face = 0
    video_capture = cv2.VideoCapture(id)
    face = None
    list_face = np.zeros(shape = (len_of_lst, 256, 256,3), dtype=np.uint8)
    while True:
        ret, frame = video_capture.read()
        img, face = mask_detect(frame)
        list_face[num_of_face] = face
        num_of_face +=1      
        # cv2.imshow("img",frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            video_capture.release()
            cv2.destroyAllWindows()
            break
    video_capture.release()
    cv2.destroyAllWindows()
    return list_face

In [122]:
id = "../train_avi/videos/1.mp4"
lsst, lst = PlayCamera(id)
cv2.destroyAllWindows()

c:\Users\ASUS\anaconda3\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


In [123]:
cv2.imshow("img",lsst[1])
cv2.waitKey(0) 
cv2.destroyAllWindows()

In [110]:
cv2.destroyAllWindows()

In [119]:
print(type(lst[0][0,0,0]))
print(type(lsst[0,0,0,0]))

<class 'numpy.uint8'>
<class 'numpy.float64'>


In [ ]:
# video_capture.release()

In [ ]:
# id = "../train/videos/1.mp4"
# video_capture = cv2.VideoCapture(id)
# num = 0
# while True:
    
#     ret, frame = video_capture.read()
#     num += 1
#     img, face = mask_detect(frame)

#     cv2.imshow('{}'.format(id), img)        
#     if cv2.waitKey(1) & 0xFF == ord('q'):
        

#         break
    
# print(num)
# PlayCamera()

In [ ]:
#Convert to AVI
# import imageio
# src_dir = "../train/videos/"
# dest_dir = "../train_avi/videos/"
# lstmp4 = os.listdir(src_dir)

# for file in lstmp4:
#     scr = src_dir+file
    
#     des = dest_dir + file
    
#     reader = imageio.get_reader(scr)
#     fps = reader.get_meta_data()['fps']
#     writer = imageio.get_writer(des, fps=fps)

#     for im in reader:
#         writer.append_data(im[:, :, :])
#     writer.close()

In [101]:
img1 = cv2.imread("Truyen_small.png")
img2 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(img1,100,200)
img_zero = np.zeros(shape =(img1.shape[0],img1.shape[1],3))
x = cosine_similarity(np.array([lsst[1].reshape(-1)]), np.array([lsst[59].reshape(-1)]))


In [102]:
print(x)

[[0.90213471]]


In [105]:
print(lst[0][0,1])
print(lsst[0,0,1])

[22 49 34]
[22. 49. 34.]


In [16]:
print(img1.shape)

(591, 443, 3)


In [15]:
cv2.imshow("x",edges)
cv2.waitKey(0)
cv2.destroyAllWindows()